In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import time

import numpy as np
import pandas as pd
import bloscpack as bp
import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from annotation_utilities import *
from registration_utilities import *
from metadata import *
from data_manager import *
from vis3d_utilities import *

Setting environment for Local Macbook Pro


Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.


/home/yuncong/CSHL_data_processed/MD585/MD585_prep2_thumbnail/MD585-N16-2015.07.16-20.32.33_MD585_2_0047_prep2_thumbnail.tif
/home/yuncong/CSHL_data_processed/MD589/MD589_prep2_thumbnail/MD589-N16-2015.07.30-17.03.43_MD589_3_0048_prep2_thumbnail.tif
/home/yuncong/CSHL_data_processed/MD590/MD590_prep2_thumbnail/MD590-IHC17-2015.08.10-19.09.09_MD590_2_0050_prep2_thumbnail.tif
/home/yuncong/CSHL_data_processed/MD591/MD591_prep2_thumbnail/MD591-IHC17-2015.08.28-04.23.47_MD591_1_0049_prep2_thumbnail.tif
/home/yuncong/CSHL_data_processed/MD592/MD592_prep2_thumbnail/MD592-N16-2015.08.22-00.50.39_MD592_2_0047_prep2_thumbnail.tif
/home/yuncong/CSHL_data_processed/MD593/MD593_prep2_thumbnail/MD593-N15-2015.08.21-16.53.02_MD593_3_0045_prep2_thumbnail.tif
/home/yuncong/CSHL_data_processed/MD594/MD594_prep2_thumbnail/MD594-IHC16-2015.08.26-16.11.01_MD594_1_0046_prep2_thumbnail.tif
/home/yuncong/CSHL_data_processed/MD595/MD595_prep2_thumbnail/MD595-IHC12-2015.09.15-01.21.39_MD595_2_0035_prep2_thumbn

In [255]:
stack = 'MD585'

In [256]:
from skimage.draw import polygon_perimeter
tb_volume, tb_volume_origin_wrt_wholebrain = DataManager.load_intensity_volume_v3(stack=stack, prep_id=4, return_origin_instead_of_bbox=True)
# tb_volume = tb_volume.astype(np.float32)

In [ ]:
# tb_volume_sectioned_rgb = np.rollaxis(np.array([tb_volume, tb_volume, tb_volume]), 0, 4)
# tb_volume_sectioned_alpha = int(255*0.01) * np.any(tb_volume_sectioned_rgb, axis=-1).astype(np.uint8)

In [257]:
tb_volume_sectioned_rgb = np.zeros(tb_volume.shape + (3,), np.uint8)
tb_volume_sectioned_alpha = np.zeros(tb_volume.shape, np.uint8)

In [258]:
start_sec = 166
secmin, secmax = metadata_cache['section_limits'][stack]
start_z = int(np.round((start_sec - secmin) / float(secmax - secmin) * tb_volume_sectioned_rgb.shape[2]))
print 'start_z =', start_z

start_z = 113


In [259]:
interval = 300
thickness = 1
# interval = 1
# thickness = 1

# for z in range(start_z, tb_volume_sectioned_rgb.shape[2], interval):
for z in [start_z, 280]:
    tb_volume_sectioned_rgb[:, :, z:z+thickness] = tb_volume[:, :, z:z+thickness, None]
    
    tb_volume_sectioned_rgb[:, xmin, z] = 255
    tb_volume_sectioned_rgb[:, -1, z] = 255
    tb_volume_sectioned_rgb[ymin, :, z] = 255
    tb_volume_sectioned_rgb[-1, :, z] = 255
    
    tb_volume_sectioned_alpha[:, :, z:z+thickness] = (5 * (tb_volume[:, :, z:z+thickness] > 0)).astype(np.uint8)
    
    tb_volume_sectioned_alpha[:, xmin, z:z+thickness] = 255
    tb_volume_sectioned_alpha[:, -1, z:z+thickness] = 255
    tb_volume_sectioned_alpha[ymin, :, z] = 255
    tb_volume_sectioned_alpha[-1, :, z] = 255

In [260]:
# Draw human annotation contours on the thumbnail volume.

contour_df = DataManager.load_annotation_v4(stack=stack, by_human=True, timestamp='latest', suffix='contours')
contour_df = convert_annotation_v3_original_to_aligned(contour_df, stack=stack)
# contour_df vertices wrt to aligned uncropped lossless image aka "wholebrain"

aws s3 cp --recursive "s3://mousebrainatlas-data/CSHL_labelings_v3/MD585" "/home/yuncong/CSHL_labelings_v3/MD585" --exclude "*" --include "*contours*"
latest timestamp:  08012017212649


In [261]:
for name_s in all_known_structures_sided:
# for name_s in ['SC']:
    
    try:

        name_u, side = parse_label(name_s, singular_as_s=True)[:2]

        for cid, cnt in contour_df[(contour_df['name'] == name_u) & (contour_df['side'] == side)].iterrows():

            contour_xysec_wrt_wholebrain_rawResol = [(x, y, cnt['section']) for x, y in cnt['vertices']]

            contour_xyz_wrt_wholebrain_volResol = \
            convert_frame_and_resolution(contour_xysec_wrt_wholebrain_rawResol,
                                          in_wrt='wholebrain', in_resolution='image_image_section',
                                          out_wrt='wholebrain', out_resolution='volume',
                                         image_resolution='raw', stack=stack,
                                         volume_resolution_um=convert_resolution_string_to_um(resolution='down32', stack=stack))

            contour_xyz_wrt_brainstemXYFullNoMargin_volResol = contour_xyz_wrt_wholebrain_volResol - tb_volume_origin_wrt_wholebrain

            z_wrt_brainstemXYFullNoMargin_volResol = min(contour_xyz_wrt_brainstemXYFullNoMargin_volResol[0,2].astype(np.int), tb_volume.shape[2]-1)

            if z_wrt_brainstemXYFullNoMargin_volResol == start_z or z_wrt_brainstemXYFullNoMargin_volResol % 7 == 0:

                polygon_ys_wrt_brainstemXYFullNoMargin_volResol, \
                polygon_xs_wrt_brainstemXYFullNoMargin_volResol = \
                polygon_perimeter(contour_xyz_wrt_brainstemXYFullNoMargin_volResol[:,1], contour_xyz_wrt_brainstemXYFullNoMargin_volResol[:,0])

                tb_volume_sectioned_rgb[polygon_ys_wrt_brainstemXYFullNoMargin_volResol.astype(np.int),
                                        polygon_xs_wrt_brainstemXYFullNoMargin_volResol.astype(np.int),
                                        z_wrt_brainstemXYFullNoMargin_volResol] = name_unsided_to_color[convert_to_unsided_label(name_s)]

                tb_volume_sectioned_alpha[polygon_ys_wrt_brainstemXYFullNoMargin_volResol.astype(np.int),
                                        polygon_xs_wrt_brainstemXYFullNoMargin_volResol.astype(np.int),
                                        z_wrt_brainstemXYFullNoMargin_volResol] = 50

    except:
        sys.stderr.write("Error reconstructing %s\n" % name_s)
        continue

In [262]:
xmin = 520
ymin = 80

In [263]:
tb_volume_sectioned_with_contours_actor = actor_volume_v2(tb_volume_sectioned_rgb[ymin:, xmin:, start_z:], 
                                                          origin=tb_volume_origin_wrt_wholebrain,
                                                          alpha=tb_volume_sectioned_alpha[ymin:, xmin:, start_z:])

In [264]:
launch_vtk([tb_volume_sectioned_with_contours_actor], init_angle='45')